In [ ]:
pip install transformers torch

In [ ]:
import yfinance as yf
import pandas as pd

# List of 10 stock tickers (you can change them)
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "TSLA",
           "MC.PA", "TTE.PA", "SAN.PA", "AIR.PA", "SU.PA"]

# Dictionary to store all data
all_data = {}

# Loop through tickers
for ticker_symbol in tickers:
    ticker = yf.Ticker(ticker_symbol)
    # Fetch historical data for 1 month with 1-hour intervals
    data = ticker.history(period="1mo", interval="1h")
    all_data[ticker_symbol] = data
    print(f"Data for {ticker_symbol}:")
    print(data.head(), "\n")  # show first few rows

# Optionally, combine into a single DataFrame
combined_data = pd.concat(all_data, axis=1)
print(combined_data.head())

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Your API key
API_KEY = "c682a0eeff664f1b83715929492c7007"

# List of 10 companies/tickers
companies = ["AAPL", "MSFT", "AMZN", "GOOGL", "TSLA",
           "MC.PA", "TTE.PA", "SAN.PA", "AIR.PA", "SU.PA"]

# Date range: last month
end_date = datetime.today()
start_date = end_date - timedelta(days=30)
from_date = start_date.strftime("%Y-%m-%d")
to_date = end_date.strftime("%Y-%m-%d")

# Prepare a list to store all articles
all_articles = []

# Loop through each company
for company in companies:
    url = (
    f"https://newsapi.org/v2/everything?"
    f"q={company}&"
    f"from={from_date}&"
    f"to={to_date}&"
    f"sortBy=publishedAt&"
    f"pageSize=100&"
    f"language=en&"  # <-- filter English articles
    f"apiKey={API_KEY}"
    )

    response = requests.get(url)
    data = response.json()

    # Check if articles exist
    articles = data.get("articles", [])
    for article in articles:
        all_articles.append({
            "Company": company,
            "Title": article.get("title"),
            "Summary": article.get("description"),
            "URL": article.get("url"),
            "PublishedAt": article.get("publishedAt")
        })

# Convert to DataFrame
df = pd.DataFrame(all_articles)

# Display the table
print(df.head(20))  # show first 20 rows
print(f"Total articles fetched: {len(df)}")

   Company                                              Title  \
0     AAPL  Belkin、MagSafe/Qi2 25Wワイヤレス充電に対応したiPhone Air/1...   
1     AAPL  Stock market today: Dow, S&P 500, Nasdaq climb...   
2     AAPL  Stock market today: Dow, S&P 500, Nasdaq slip ...   
3     AAPL  Parallels、macOS 26 TahoeやWindows 11 25H2をサポートし...   
4     AAPL          Apple M5チップを搭載した「MacBook Pro」が近く発表されるもよう。   
5     AAPL  Sustained iPhone demand points to 2026 upside ...   
6     AAPL  Dear Salesforce Stock Fans, Mark Your Calendar...   
7     AAPL  Stocks Turn Mixed as Walmart-OpenAI Deal Pushe...   
8     AAPL  Here's How Much $100 Invested In Apple 15 Year...   
9     AAPL   Stocks Fall as US-China Trade Conflict Escalates   
10    AAPL                              rustybt added to PyPI   
11    AAPL  Friday’s selloff broke something in the stock ...   
12    AAPL  Global markets tumble as Beijing imposes new b...   
13    AAPL  Global markets tumble as Beijing imposes new b...   
14    AAPL  Nonstrict、Air

In [ ]:
# Convert all summaries to strings and replace NaN with empty string
df["Summary"] = df["Summary"].fillna("").astype(str)

# Optional: remove articles with empty summaries entirely
df = df[df["Summary"].str.strip() != ""].reset_index(drop=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# FinBERT pré-entraîné pour sentiment financier
MODEL_NAME = "yiyanghkust/finbert-tone"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
def analyze_sentiment(text):
    # Tokenize le texte
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Passage dans le modèle
    outputs = model(**inputs)

    # Softmax pour obtenir les probabilités
    probs = F.softmax(outputs.logits, dim=1)

    # Indices et labels
    sentiment_labels = ["positive", "neutral", "negative"]
    sentiment_idx = torch.argmax(probs, dim=1).item()

    # Probabilité du label choisi
    sentiment_score = probs[0][sentiment_idx].item()

    return sentiment_labels[sentiment_idx], sentiment_score


In [ ]:
sentiments = [analyze_sentiment(text) for text in df["Summary"]]

df["Sentiment"] = [s[0] for s in sentiments]
df["SentimentScore"] = [s[1] for s in sentiments]


# Show results
print(df.head(20))


   Company                                              Title  \
0     AAPL  Belkin、MagSafe/Qi2 25Wワイヤレス充電に対応したiPhone Air/1...   
1     AAPL  Stock market today: Dow, S&P 500, Nasdaq climb...   
2     AAPL  Stock market today: Dow, S&P 500, Nasdaq slip ...   
3     AAPL  Parallels、macOS 26 TahoeやWindows 11 25H2をサポートし...   
4     AAPL          Apple M5チップを搭載した「MacBook Pro」が近く発表されるもよう。   
5     AAPL  Sustained iPhone demand points to 2026 upside ...   
6     AAPL  Dear Salesforce Stock Fans, Mark Your Calendar...   
7     AAPL  Stocks Turn Mixed as Walmart-OpenAI Deal Pushe...   
8     AAPL  Here's How Much $100 Invested In Apple 15 Year...   
9     AAPL   Stocks Fall as US-China Trade Conflict Escalates   
10    AAPL                              rustybt added to PyPI   
11    AAPL  Friday’s selloff broke something in the stock ...   
12    AAPL  Global markets tumble as Beijing imposes new b...   
13    AAPL  Global markets tumble as Beijing imposes new b...   
14    AAPL  Nonstrict、Air